In [1]:
import numpy as np
import pandas as pd
from googleapiclient.discovery import build

In [8]:
# Youtube Data API:
yt = build(
    serviceName = "youtube",
    version = "v3",
    developerKey = "" # my key
)

In [9]:
### Channels sample

# Ids from (content = "https://www.youtube.com/channel/...) and titles:
channels = {
    "UCYO_jab_esuFRV4b17AJtAw": "3Blue1Brown",
    "UCNKcMBYP_-18FLgk4BYGtfw": "AWE me",
    "UCiDJtJKMICpb9B1qf7qjEOA": "Adam Savage’s Tested",
    "UC32z4mtyiq02Ge-XWy78ibw": "Aero Por Trás da Aviação",
    "UCaRMivfyupj3ucUyJbZbCNg": "Anglo-Link",
    "UCwg6_F2hDHYrqbNSGjmar4w": "Animalogic",
    "UCi5iiEyLwSLvlqnMi02u5gQ": "Ask a Mortician",
    "UC-9b7aDP6ZN0coj9-xFnrtw": "Astrum",
    "UCSTlOTcyUmzvhQi6F8lFi5w": "Atila Iamarino",
    "UCLXl1V6n82Dyg1VhVgSL0nw": "Aviões e Músicas com Lito Sousa",
    "UCGlmclFVnJCFQ_VO4kTgTig": "Azusa Barbie",
    "UCCj956IF62FbT7Gouszaj9w": "BBC",
    "UCxcnsr1R5Ge_fbTu5ajt8DQ": "Bob Ross",
    "UCGp4u0WHLsK8OAxnvwiTyhA": "British Pathé",
    "UCQRPDZMSwXFEDS67uc7kIdg": "Buenas Ideias",
    "UCcyq283he07B7_KUX07mmtA": "Business Insider",
    "UCvJJ_dzjViJCoLf5uKUTwoA": "CNBC",
    "UCWq1xltHB2fDe6YkYoOrryg": "Canal do Schwarza",
    "UCwnKziETDbHJtx78nIkfYug": "CaspianReport",
    "UCR1-7g_y2YcYlh1W9y_1LUg": "Chaves Estranho",
    "UC04KsGq3npibMCE9Td3mVDg": "Cheddar",
    "UCpVm7bg6pXKo1Pr6k5kxG9A": "Classic Mr Bean",
    "UCvmijL-eepDVHYSJHDY3d6w": "Cole and Marmalade",
    "UC9-y-6csu5WGm29I7JiwpnA": "Computerphile",
    "UCyAMV63OY0DRbKk6gGv6wJg": "Cradle Of Filth",
    "UC726J5A0LLFRxQ0SZqr2mYQ": "Curious Droid",
    "UCFuIUoyHB12qpYa8Jpxoxow": "Código Fonte TV",
    "UCd-swDW3HCs4LTZhq8Cf7Fg": "DW Euromaxx",
    "UCa37IMrH8BGS_pO6CKbTL-A": "Dinosaurs",
    "UC_5niPa-d35gg88HaS7RrIw": "Disney",
    "UC1xwwLwm6WSMbUn_Tp597hQ": "Disney Parks",
    "UCVRrGAcUc7cblUzOhI1KfFg": "Earthling Ed",
    "UCIBaDdAbGlFDeS33shmlD0A": "European Space Agency, ESA",
    "UC6uKrU_WqJ1R2HMTY3LIx5Q": "Everyday Astronaut",
    "UCZ6JHFBaDUJ9wfo41HSII_w": "Fabio Chaves",
    "UC6zbvGt_jJVgw7-MWkVUC5A": "Fala Vegan",
    "UCxGiUjep8KiihvHldRousPA": "Fancy Fairy Wings & Things",
    "UC_qKzDKiOkCjkz0JKl1BufA": "Gary L. Francione",
    "UCeSRjhfeeqIgr--AcP9qhyg": "Guinness World Records",
    "UCLAcytNR3gdw44yzoSiKluA": "Hello Korea",
    "UC26YLK0OEbLB3TCYxGh8xVQ": "Huygens Optics",
    "UCcMDMoNu66_1Hwi5-MeiQgw": "Hydraulic Press Channel",
    "UCkAbeyJZqp6JFYfKy-98NjA": "Integrando Conhecimento",
    "UCJ8bjM5yQSkOP-_99n1zKkw": "Intensivo Pedagógico",
    "UCZ9l_6_f0PWRYXN5Y7Lcl2A": "Jacques Slade",
    "UCdGkmb5zEDXnPSmQlV43t0Q": "JamesRandiFoundation",
    "UCbbQalJ4OaC0oQ0AqRaOJ9g": "Jay Foreman",
    "UCkP2CvRubyU0MTZv_Qo712g": "Jessica in the Kitchen - Vegan Recipes",
    "UC-2YHgc363EdcusLIBbgxzg": "Joe Scott",
    "UCBbnbBWJtwsf0jLGUwX5Q3g": "Journey to the Microcosmos",
    "UCLQWhXC5aOZ_tQNloBg40BA": "Kirsten & Joerg",
    "UCuCuEKq1xuRA0dFQj1qg9-Q": "Knowledgia",
    "UCsXVk37bltHxD1rDPwtNM8Q": "Kurzgesagt – In a Nutshell",
    "UCNqNkZ7kKfqimqHkgbWMNYA": "Launch Pad Astronomy",
    "UCoNTMWgGuXtGPLv9UeJZwBw": "Living Big In A Tiny House",
    "UCt2WVZXVrHaFgZ432s33fLw": "MAKE UP FOR EVER",
    "UCBlXovStrlQkVA2xJEROUNg": "Marcel Vos",
    "UCr22xikWUK2yUW4YxOKXclQ": "MathTheBeautiful",
    "UC6nZXhOz_gHr4mlb5UNl9ug": "Matt Jordan",
    "UCqsyr4vgV0tWxo0jgeoOZjg": "MechanicalFiend",
    "UCQwFuQLnLocj5F7ZcmcuWYQ": "MetaBallStudios",
    "UCk5BcU1rOy6hepflk7_q_Pw": "Meteoro Brasil",
    "UCqONzeACDBaF6FfKjh7ndAQ": "Microsoft Flight Simulator",
    "UChHS4NI6U4XgCuYgsrygVCA": "Mintfaery",
    "UC1ZBQ-F-yktYD4m5AzM6pww": "Mustard",
    "UCLA_DiR1FfKNvjuUpBHmylQ": "NASA",
    "UCeY0bbntWzzVIaj2z3QigXg": "NBC News",
    "UCPZvqkyXoYDlzBClxpxVIzg": "Naomi Farr",
    "UCggHsHce2n3vvbJf_8YKrMA": "Nerdforge",
    "UCWOA1ZGywLbqmigxE4Qlvuw": "Netflix",
    "UCD57tGnYPW1twCohgIVkJpw": "Nox et Lux",
    "UCoxcjq-8xIDTYp3uz647V5A": "Numberphile",
    "UCroDJPcFCf6DBmHns6Xeb8g": "Nyma Tang",
    "UCQ3JxE-NOyZaJ3m3qIkZbhA": "O Pimentinha",
    "UCtb2f3j_sB4Z4Ik2YJtbzsg": "OneikaTraveller",
    "UCwK-svlNEASA7g_smu1_d8g": "OnlyConnect Fan",
    "UCpLQXR116cLVUa1LRY8KS4w": "OwlKitty",
    "UCzR-rom72PHN9Zg7RML9EbA": "PBS Eons",
    "UC7_gcs09iThXybpVgjHZ_7g": "PBS Space Time",
    "UCddYq41_tZ1FnLlguLT6-Ow": "Parafernalha",
    "UCOVkkVaUP0Xop6IIjyk86QA": "Perception",
    "UCtESv1e7ntJaLJYKIO1FoYw": "Periodic Videos",
    "UC7DdEm33SyaTDtWYGO2CwdA": "Physics Girl",
    "UCQwHYU3ZbSLG6LOVwtvuozg": "Planeta Aves",
    "UCEWHPFNilsT0IfQfutVzsag": "Porta dos Fundos",
    "UClo-U5gvXPRy-4VqWEkkRDg": "Primitive Building",
    "UCKKJpBveT8vWVNfLQ-MvZMg": "Prof. André Azevedo da Fonseca",
    "UCXOzDw_X92bNqSCqw-NojiA": "Professor Julio Borbo",
    "UCwSxSJqGpSRpEsq5-YUbM8g": "Professor Noslen",
    "UCNzul4dnciIlDg8BAcn5-cQ": "Prowalk Tours",
    "UCvn_XCl_mgQmt3sD753zdJA": "Rachel's English",
    "UCR1IuLEqb6UEA_zQ81kwXfg": "Real Engineering",
    "UCP5tjEmvPItGyLhmjdwP7Ww": "RealLifeLore",
    "UCFJxE0l3cVYU4kHzi4qVEkw": "Rebecca Watson",
    "UCliwOcLaEfqkGJ7xq2ipq4w": "Reverse Engineering",
    "UCovtFObhY9NypXcyHxAS7-Q": "Roberto Blake",
    "UCZYTClx2T1of7BRZ86-8fow": "SciShow",
    "UCxzC4EngIsMrPmbm6Nxvb-A": "Scott Manley",
    "UC3KEoMzNz8eYnwBC34RaKCQ": "Simone Giertz",
    "UCT3wwJBh8fwePiNC63NlzCA": "Simple Flying",
    "UC6107grRI4m0o2-emgoDnAA": "SmarterEveryDay",
    "UC_Fk7hHbl7vv_7K8tYqJd5A": "SpaceToday",
    "UCtI0Hodo5o5dUb67FeUjDeA": "SpaceX",
    "UCZ9jWH_8tJ-Nmaj8dSQdEYA": "Stefan Milo",
    "UCpQLC-evmUAon9BBpcW4kYg": "Steve Cutts",
    "UC6u6uY4VbvuNtU0BU7F9olw": "Studson Studio",
    "UCRXRbi80k0_vcIfgpOSerTg": "Subject Zero Science",
    "UCAuUUnT6oDeKwE6v1NGQxug": "TED",
    "UChDKyKQ59fYz3JO2fl0Z6sg": "TODAY",
    "UCZdJE8KpuFm6NRafHTEIC-g": "Tempero Drag",
    "UC6n8I1UDTKP1IWjQMg6_TwA": "The B1M",
    "UCvpQ-l09fCVxJd3urZbxzHg": "The British Museum",
    "UCsQXQMAZTygXddj_I4Ud9xA": "The Green Witch",
    "UCUK0HBIBWgM2c4vsPhkYY4w": "The Slow Mo Guys",
    "UC2LVhJH_9cT2XKp0VAfsKOQ": "The Tim Traveller",
    "UCBa659QWEk1AI4Tg--mrJ2A": "Tom Scott",
    "UClVbhSLxwws-KSsPKz135bw": "TopMovieClips",
    "UC29ju8bIPH5as8OGnQzwJyA": "Traversy Media",
    "UCgtq_tKnFFQyJ-7j0yr9utg": "Tyta Montrase",
    "UCSxFcMDGmwgDlSP9qmYVx1w": "TÁ NA MESA VEGG",
    "UCVEVuanoMK9tGclfWLghaKw": "Tá Querida",
    "UCTlLFaGX4q2otO0ZVxLW0VQ": "Uma história a mais",
    "UC9fSZHEh6XsRpX-xJc6lT3A": "UrAvgConsumer",
    "UCp3iXxis9n_E_GfbE-_ksFw": "VeganBlackMetalChef",
    "UCHnyfMqiRRG1u-2MsSQLbXA": "Veritasium",
    "UCYulrOk4EbqL833oETw0bWQ": "Wanderlust Travel Videos",
    "UCaWd5_7JhbQBe4dknZhsHJg": "WatchMojo.com",
    "UCmVa-cbCpkd5Cd9Fr_4tCWg": "Your Dinosaurs Are Wrong",
    "UCXvA7wLxIJDa1SWYWl7e-oQ": "arewethereyet",
    "UCS4iXNXsvlOHm7TuXLltreg": "chilli music",
    "UC9Ntx-EF3LzKY1nQ5rTUP2g": "cyriak",
    "UCmEKnMzbltaFyiA6H46IDng": "tatianagfeltrin",
    "UCfwx98Wty7LhdlkxL5PZyLA": "truly",
    "UCVpankR4HtoAVtYnFDUieYA": "zefrank1"
 }

In [10]:
len(channels)

134

In [11]:
###### Videos data

# Loop in the channels:
df_videos = pd.DataFrame(
    {
        "channel_title": pd.Series([], dtype = "str"),
        "channel_id": pd.Series([], dtype = "str"),
        "video_title": pd.Series([], dtype = "str"),
        "video_id": pd.Series([], dtype = "str"),
        "video_upload_date": pd.Series([], dtype = "str"),
        "views": pd.Series([], dtype = "int"),
        "likes": pd.Series([], dtype = "int"),
        "dislikes": pd.Series([], dtype = "int"),
        "comments": pd.Series([], dtype = "int")
    }
)
for chnl_ind, chnl_key in enumerate(channels):

    # Choose the channel:
    channel_id = chnl_key
    channel_title = channels[channel_id]

    ### Extract the data from all the videos of the channel

    # Content details:
    content = yt.channels().list(
        id = channel_id,
        part = "contentDetails"
    ).execute()

    # Upload Id:
    upload_id = content["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # All the channel's videos:
    all_videos = []
    next_pg_token = None
    res = yt.playlistItems().list(
        playlistId = upload_id,
        maxResults = 50,
        part = "snippet",
        pageToken = next_pg_token
    ).execute()
    while True:
        res = yt.playlistItems().list(
            playlistId = upload_id,
            maxResults = 50,
            part = "snippet",
            pageToken = next_pg_token
        ).execute()
        all_videos += res["items"]
        next_pg_token = res.get("nextPageToken")
        if next_pg_token is None:
            break

    # Videos statistics:
    videos_ids = list(map(lambda x: x["snippet"]["resourceId"]["videoId"], all_videos))
    stats = []
    for i in range(0, len(videos_ids), 40):
        res = (yt).videos().list(
            id = ",".join(videos_ids[i:i + 40]),
            part = "statistics"
        ).execute()
        stats += res["items"]

    # Put it in a dataframe:
    video_id, views, likes, dislikes, comments = [], [], [], [], []
    for i in range(len(stats)):
        video_id += [stats[i]["id"]]
        stats_i = stats[i]["statistics"]
        views += [stats_i[k] if "viewCount" in stats_i.keys() else np.nan for k in ["viewCount"]]
        likes += [stats_i[k] if "likeCount" in stats_i.keys() else np.nan for k in ["likeCount"]]
        dislikes += [stats_i[k] if "dislikeCount" in stats_i.keys() else np.nan for k in ["dislikeCount"]]
        comments += [stats_i[k] if "commentCount" in stats_i.keys() else np.nan for k in ["commentCount"]]
    df_channel_videos = pd.DataFrame(
        {
            "channel_title": channel_title,
            "channel_id": channel_id,
            "video_title": list(map(lambda x: x["snippet"]["title"], all_videos)),
            "video_id": video_id,
            "video_upload_date": list(map(lambda x: x["snippet"]["publishedAt"], all_videos)),
            "views": views,
            "likes": likes,
            "dislikes": dislikes,
            "comments": comments        
        }
    )
    df_videos = pd.concat([df_videos, df_channel_videos])

# Save the dataframe:
df_videos.to_csv("data/videos_data.csv",
                sep = ";",
                index = False)


channel: 3Blue1Brown
channel: AWE me
channel: Adam Savage’s Tested
channel: Aero Por Trás da Aviação
channel: Anglo-Link
channel: Animalogic
channel: Ask a Mortician
channel: Astrum
channel: Atila Iamarino
channel: Aviões e Músicas com Lito Sousa
channel: Azusa Barbie
channel: BBC
channel: Bob Ross
channel: British Pathé
channel: Buenas Ideias
channel: Business Insider
channel: CNBC
channel: Canal do Schwarza
channel: CaspianReport
channel: Chaves Estranho
channel: Cheddar
channel: Classic Mr Bean
channel: Cole and Marmalade
channel: Computerphile
channel: Cradle Of Filth
channel: Curious Droid
channel: Código Fonte TV
channel: DW Euromaxx
channel: Dinosaurs
channel: Disney
channel: Disney Parks
channel: Earthling Ed
channel: European Space Agency, ESA
channel: Everyday Astronaut
channel: Fabio Chaves
channel: Fala Vegan
channel: Fancy Fairy Wings & Things
channel: Gary L. Francione
channel: Guinness World Records
channel: Hello Korea
channel: Huygens Optics
channel: Hydraulic Press Ch

In [12]:
df_videos

,channel_title,channel_id,video_title,video_id,video_upload_date,views,likes,dislikes,comments
0,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A few of the best math explainers from this su...,F3Qixy-r_rQ,2021-10-23T18:11:23Z,361195,24407,82,659
1,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,How a Mandelbrot set arises from Newton’s work,LqbZpur38nw,2021-10-15T16:41:50Z,554691,25722,144,1205
2,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,Newton's Fractal (which Newton knew nothing ab...,-RdOwhmqP5s,2021-10-07T02:19:39Z,1179922,61162,285,2792
3,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,The Summer of Math Exposition,ojjzXyQCzso,2021-07-16T15:37:16Z,609396,29267,214,1719
4,3Blue1Brown,UCYO_jab_esuFRV4b17AJtAw,A quick trick for computing eigenvalues | Chap...,e50Bj7jn9IQ,2021-05-07T19:01:16Z,419586,16984,144,1145
...,...,...,...,...,...,...,...,...,...
191,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Hole,LPfcGXMpKds,2009-06-06T22:16:56Z,90791,1791,23,75
192,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Affordable Button,h3lvSflNixI,2009-05-30T00:13:46Z,103154,1566,47,18
193,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Optical Illusion,8mS5RK0Yo6w,2009-05-23T22:36:58Z,1072148,9848,1201,NaN
194,zefrank1,UCVpankR4HtoAVtYnFDUieYA,HardTimes :: Outsource,zxIbQ6ZG4OI,2009-05-17T00:50:31Z,78193,1620,20,87
